<a href="https://colab.research.google.com/github/MagnusRedkin/Netology/blob/main/%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_%D0%BA_%D0%B7%D0%B0%D0%BD%D1%8F%D1%82%D0%B8%D1%8E_%C2%AB%D0%9A%D0%BB%D0%B0%D1%81%D1%81%D0%B8%D1%84%D0%B8%D0%BA%D0%B0%D1%86%D0%B8%D1%8F_%D0%9B%D0%BE%D0%B3%D0%B8%D1%81%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D1%80%D0%B5%D0%B3%D1%80%D0%B5%D1%81%D1%81%D0%B8%D1%8F_%D0%B8_SVM%C2%BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание
Решить задачу классификации физических лиц по уровню дохода. Данные для обучения модели хранятся в файле adult.csv, который можно найти в материалах к занятию или скачать с сайта.

Целевая переменная – уровень дохода income, который принимает два значения <=50K и >50K, поэтому классификация бинарная. Остальные признаки описывают персональную информацию – возраст, образование, семейное положение и т. д. Подробное описание признаков и их возможные значения можно получить самостоятельно, используя функции Python3 для анализа датасета (describe, unique и т.д) или прочитать информацию по ссылке. http://www.cs.toronto.edu/~delve/data/adult/adultDetail.html

Задачу классификации нужно решить при помощи обучения модели логистической регрессии и модели опорных векторов.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

In [2]:
data = pd.read_csv('https://github.com/MagnusRedkin/Netology/raw/main/adult.csv')
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        48842 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       48842 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   48842 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


Проверим данные на пропуски.

In [4]:
data.isna().sum()

age                0
workclass          0
fnlwgt             0
education          0
educational-num    0
marital-status     0
occupation         0
relationship       0
race               0
gender             0
capital-gain       0
capital-loss       0
hours-per-week     0
native-country     0
income             0
dtype: int64

Пропусков нет.

Выбрем признаки.

Оставим все числа, потому что с нними проще, кроме fnlwgt,
и выберем объекты. 

Как удалось выяснить, fnlwgt - примерное количество человек, к которым относится данная строка.
(кривоват перевод получился), т.е. количество человек с одинаковыми признаками. Для поиска уровеня дохода, такая характеристика не нужна, а внести путаницу может.

education не берём, так как есть educational-num.

marital-status тоже, так как не царская Россия, когда женатым платили больше.

оставим occupation, gender и native-country. так как от специальности зависит очень сильно, женщинам и приезжим платят меньше.

In [18]:
selectedColumns = data[['age', 'gender', 'educational-num', 'occupation', 'native-country',
                        'capital-gain', 'capital-loss','hours-per-week', 'income']]

In [19]:
selectedColumns['occupation'].value_counts()

Craft-repair         6020
Prof-specialty       6008
Exec-managerial      5984
Adm-clerical         5540
Sales                5408
Other-service        4808
Machine-op-inspct    2970
Transport-moving     2316
Handlers-cleaners    2046
Farming-fishing      1480
Tech-support         1420
Protective-serv       976
Priv-house-serv       232
Armed-Forces           14
Name: occupation, dtype: int64

Вылез '?', убирать его или пройдёт за ещё одну специальность? Вначале посчитаем с ним, потом очистим.
Аналогично с native-country.

Преобразуем категориальные переменные в числа и удалим столбец являющийся целевой переменной, 

In [25]:
X = pd.get_dummies(selectedColumns, columns=['gender', 'occupation', 'native-country'], drop_first=True)
del X['income']
X.head()

,age,educational-num,capital-gain,capital-loss,hours-per-week,gender_Male,occupation_Armed-Forces,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,25,7,0,0,40,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,38,9,0,0,50,1,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2,28,12,0,0,40,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,44,10,7688,0,40,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,34,6,0,0,30,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


Переведём категориальную целевую переменную в числа.

In [26]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(data['income'])

LabelEncoder()

In [27]:
Y = pd.Series(data=le.transform(data['income']))
Y.head()

0    0
1    0
2    1
3    1
4    0
dtype: int64

Масштабируем модели.

In [28]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 
from sklearn.svm import SVC

model = make_pipeline(StandardScaler(),LogisticRegression(max_iter=1000))
model2 = make_pipeline(StandardScaler(), SVC(kernel='poly')) 

Делим на обучающую выборку и тестовую.

In [29]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=21)
X_test.index

Int64Index([12043,  3562, 33567, 36986, 13555, 24097, 14391, 11005, 42075,
            43416,
            ...
            37584, 12617, 36258, 38931, 14379, 11854, 48428,  8312,  8131,
            36096],
           dtype='int64', length=9045)

Обучаем.

In [30]:
model.fit(X_train, Y_train)
predictions = model.predict(X_test)
model2.fit(X_train, Y_train)
predictions = model2.predict(X_test)

Анализируем точность предсказания.

In [13]:
model.score(X_train, Y_train) 

0.8304199831085404

In [14]:
model.score(X_test, Y_test)

0.8280274337189067

In [15]:
model2.score(X_train, Y_train) 

0.8207457835333862

In [16]:
model2.score(X_test, Y_test)

0.8124680110553792

Разница между обучающей и тестовой базами минимальна, значит всё в порядке. Правда логистическая регрессия оказалась немного лучше модели опорных векторов.

А теперь удалим строки с "вопросительной" специальностью и провторим обучение.

In [17]:
data = data[~(data['occupation']=='?')]
data = data[~(data['native-country']=='?')]

In [31]:
model.score(X_train, Y_train) 

0.8249993089532023

In [32]:
model.score(X_test, Y_test)

0.8223327805417358

In [33]:
model2.score(X_train, Y_train) 

0.815158802554109

In [34]:
model2.score(X_test, Y_test)

0.8019900497512438

Да, очистка специальности и native-country ухудшила работу модели.

Для улучшения точности модели нужно уметь работать с отбором признаков. Для этого нужно иметь инструмент, интуиции недостаточно.

P.S. 
Постройте 1-2 графика на выбор. Визуализация должна быть основана на исследуемых данных и быть полезной (из графика можно сделать вывод об особенностях датасета/класса/признака)

Какие графики нужно было построить, не до конца понял.